# Homework #3 by Tasneem Abbasi, Myles Cook, and Keegan Smith

## Part a

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [11]:
# Load the data
auctions_df = pd.read_csv('eBayAuctions.csv')

# Preliminary data cleaning
auctions_df.dropna(inplace=True)  # Drop rows with missing values

# Display the first few rows of the dataframe
auctions_df.head()

,Category,currency,sellerRating,Duration,endDay,ClosePrice,OpenPrice,Competitive?
0,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
1,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
2,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
3,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
4,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0


In [12]:
# Drop rows with missing values
auctions_df.dropna(inplace=True)

# Convert columns to 'category' data type
auctions_df['Category'] = auctions_df['Category'].astype('category')
auctions_df['currency'] = auctions_df['currency'].astype('category')
auctions_df['endDay'] = auctions_df['endDay'].astype('category')
auctions_df['Duration'] = auctions_df['Duration'].astype('category')

# Define categorical predictors
cat_predictors = ['Category', 'currency', 'Duration', 'endDay']

# Define all predictors
predictors = ['Category', 'currency','sellerRating', 'Duration', 'endDay', 'ClosePrice', 'OpenPrice']

# Define outcome
outcome = 'Competitive?'

# Create dummy variables for categorical predictors
X_cat = pd.get_dummies(auctions_df[cat_predictors], dtype=int)

# Concatenate categorical and numerical predictors to form feature matrix
X = pd.concat([X_cat, auctions_df[['sellerRating', 'ClosePrice', 'OpenPrice']]], axis=1)

# Define target variable
y = auctions_df['Competitive?']

In [13]:
# Define the categorical variables
categorical_vars = ['Category', 'currency', 'endDay', 'Duration']

# Loop over each categorical variable
for var in categorical_vars:
    # Create a pivot table for each variable, calculating the sum of 'Competitive?' for each category
    pivot = auctions_df.pivot_table(index=var, values='Competitive?', aggfunc='sum')
    
    # Normalize the pivot table by dividing each value by the total sum
    pivot = pivot / pivot.sum()
    
    # Sort the pivot table in descending order based on 'Competitive?'
    pivot = pivot.sort_values(by='Competitive?', ascending=False)
    
    # Print the pivot table for each variable
    print(f'{var}:\n{pivot}\n')

Category:
                      Competitive?
Category                          
Music/Movie/Game          0.227955
Collectibles              0.129456
Toys/Hobbies              0.116323
Antique/Art/Craft         0.093809
SportingGoods             0.084428
Home/Garden               0.062852
Automotive                0.059099
Clothing/Accessories      0.056285
Electronics               0.041276
Jewelry                   0.028143
Books                     0.025328
Computer                  0.022514
Business/Industrial       0.011257
Coins/Stamps              0.010319
Health/Beauty             0.010319
Photography               0.010319
Pottery/Glass             0.006567
EverythingElse            0.003752

currency:
          Competitive?
currency              
US            0.629456
EUR           0.275797
GBP           0.094747

endDay:
        Competitive?
endDay              
Mon         0.346154
Sun         0.153846
Sat         0.140713
Fri         0.125704
Thu         0.114447
Tue     

# Part b

In [14]:
# Define predictors and outcome
X = df_dummies.drop('Competitive?', axis=1)
y = df_dummies['Competitive?']

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.40, random_state=42)

# Run logistic regression
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

# Predict and calculate accuracy
y_pred = model.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print(f'Accuracy with all predictors: {accuracy}')

NameError: name 'df_dummies' is not defined

# Part c

In [ ]:
X_excl_price = df_dummies.drop(['Competitive?', 'ClosePrice'], axis=1)
X_train_excl_price, X_valid_excl_price, y_train, y_valid = train_test_split(X_excl_price, y, test_size=0.40, random_state=42)

# Scale the data excluding price
X_scaled_excl_price = scaler.fit_transform(X_train_excl_price)

# Run logistic regression
model_excl_price = LogisticRegression(max_iter=10000)
model_excl_price.fit(X_scaled_excl_price, y_train)

# Predict and calculate accuracy on the validation set
X_valid_scaled_excl_price = scaler.transform(X_valid_excl_price)
y_pred_excl_price = model_excl_price.predict(X_valid_scaled_excl_price)
accuracy_excl_price = accuracy_score(y_valid, y_pred_excl_price)
print(f'Accuracy excluding price: {accuracy_excl_price}')

Accuracy excluding price: 0.6387832699619772


# Part d

In [ ]:
import statsmodels.api as sm
model_eval = sm.Logit(y_train, sm.add_constant(X_train))
model_eval = model_eval.fit()
print(model_eval.summary())

ModuleNotFoundError: No module named 'statsmodels'

The log probability of a competetive auction is increased by 0.0894 for each 1 unit (dollar) of closing price and it is statistically significant (p < 0.1)

# Part e

In [ ]:
model_regularized = LogisticRegression(penalty='l1', solver='liblinear', max_iter=10000)
model_regularized.fit(X_scaled, y_train)

# Predict and calculate accuracy on the validation set
y_pred_regularized = model_regularized.predict(X_valid_scaled)
accuracy_regularized = accuracy_score(y_valid, y_pred_regularized)
print(f'Accuracy with L1 regularization: {accuracy_regularized}')

Accuracy with L1 regularization: 0.7515842839036755


# Part f

We would recommend an auction with a duration of 5 days ending on either Friday or Monday given that those have the highest possitive coefficients, and that the auction use British Pounds (GBP) and is in the category "photography" although the reliance of these data is undetermined in our model. 